In [9]:
import polars as pl
import xmltodict

In [10]:
# Path to the XML file
xml_file = "temp/aec-mediafeed-results-detailed-preload-20499.xml"

# Read the XML file
with open(xml_file) as file:
    xml_content = file.read()

# Convert XML to a Python dictionary
data_dict = xmltodict.parse(xml_content)

In [12]:
# data_dict['MediaFeed']
# data_dict['MediaFeed']['Results']
print(len(data_dict["MediaFeed"]["Results"]["Election"]))
# data_dict['MediaFeed']['Results']["Election"][0]["House"]
print(len(data_dict["MediaFeed"]["Results"]["Election"][0]["House"]["Contests"]["Contest"]))
data_dict["MediaFeed"]["Results"]["Election"][0]["House"]["Contests"]["Contest"][0]

2
150


{'@Projected': 'true',
 'eml:ContestIdentifier': {'@Id': '101', 'eml:ContestName': 'Canberra'},
 'PollingDistrictIdentifier': {'@Id': '101',
  '@ShortCode': 'CANB',
  'Name': 'Canberra',
  'StateIdentifier': {'@Id': 'ACT'}},
 'Enrolment': {'@CloseOfRolls': '143279',
  '@Historic': '134830',
  '#text': '143279'},
 'FirstPreferences': {'@PollingPlacesReturned': '0',
  '@PollingPlacesExpected': '53',
  'Candidate': [{'eml:CandidateIdentifier': {'@Id': '28189',
     'eml:CandidateName': 'BRODTMANN, Gai'},
    'eml:AffiliationIdentifier': {'@Id': '200',
     '@ShortCode': 'ALP',
     'eml:RegisteredName': 'Australian Labor Party'},
    'BallotPosition': '1',
    'Elected': {'@Historic': 'true', '#text': 'false'},
    'Incumbent': {'@Notional': 'true', '#text': 'true'},
    'Votes': {'@Historic': '51456',
     '@Percentage': '0',
     '@Swing': '0',
     '@MatchedHistoric': '0',
     '#text': '0'},
    'VotesByType': {'Votes': [{'@Type': 'Ordinary',
       '@Historic': '45943',
       '@Perc

In [13]:
# (
#     pl.DataFrame(data_dict['MediaFeed']['Results']["Election"][0]["House"]['Contests']['Contest'][0], strict = False)
#     .select(["FirstPreferences"])
#     .unnest("FirstPreferences")
#     .select("Candidate")
#     .explode('Candidate')
#     .unnest("Candidate")
#     .select(
#         pl.col("eml:AffiliationIdentifier").struct[2],
#         pl.col("VotesByType").struct[0]
#     )
#     .explode("Votes")
#     .unnest('Votes')
# )

In [14]:
(
    pl.DataFrame(data_dict["MediaFeed"]["Results"]["Election"][0]["House"]["Contests"]["Contest"][0], strict=False)
    .select(pl.col("PollingPlaces").struct[0].explode().alias("PollingPlace"))
    .select(
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Id"].alias("poling_place_id"),
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Name"].alias("poling_place_name"),
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Classification"].alias("poling_place_type"),
        pl.col("PollingPlace").struct["FirstPreferences"].struct["Candidate"],
    )
    .explode("Candidate")
    .with_columns(
        pl.col("Candidate").struct["eml:CandidateIdentifier"].struct[0].alias("candidate"),
        pl.col("Candidate").struct["Votes"].struct["@Historic"].alias("votes_past"),
        pl.col("Candidate").struct["Votes"].struct["@Percentage"].alias("vote_percent"),
        pl.col("Candidate").struct["Votes"].struct["@Swing"].alias("vote_swing"),
        pl.col("Candidate").struct["Votes"].struct["#text"].alias("vote_current"),
    )
    .drop("Candidate")
)

poling_place_id,poling_place_name,poling_place_type,candidate,votes_past,vote_percent,vote_swing,vote_current
str,str,str,str,str,str,str,str
"""8829""","""Barton""",null,"""28189""","""796""","""0""","""0""","""0"""
"""8829""","""Barton""",null,"""28336""","""338""","""0""","""0""","""0"""
"""8829""","""Barton""",null,"""29373""","""88""","""0""","""0""","""0"""
"""8829""","""Barton""",null,"""28989""","""742""","""0""","""0""","""0"""
"""64583""","""Belconnen CANBERRA PPVC""","""PrePollVotingCentre""","""28189""","""395""","""0""","""0""","""0"""
…,…,…,…,…,…,…,…
"""32705""","""Woden CANBERRA PPVC""","""PrePollVotingCentre""","""28989""","""5190""","""0""","""0""","""0"""
"""8840""","""Yarralumla""",null,"""28189""","""571""","""0""","""0""","""0"""
"""8840""","""Yarralumla""",null,"""28336""","""216""","""0""","""0""","""0"""


In [15]:
(
    pl.DataFrame(
        data_dict["MediaFeed"]["Results"]["Election"][0]["House"]["Contests"]["Contest"][0], strict=False
    ).select(
        pl.col("PollingDistrictIdentifier").struct["Name"].alias("electorate"),
        pl.col("PollingDistrictIdentifier").struct["StateIdentifier"].struct["@Id"].alias("state"),
    )
    # .unnest('PollingDistrictIdentifier')
    # .select()
    # .unnest('StateIdentifier')
    # .select(["FirstPreferences"])
    # .unnest("FirstPreferences")
    # .select("Candidate")
    # .explode('Candidate')
    # .unnest("Candidate")
    # .select(
    #     pl.col("eml:AffiliationIdentifier").struct[2],
    #     pl.col("VotesByType").struct[0]
    # )
    # .explode("Votes")
    # .unnest('Votes')
)

electorate,state
str,str
"""Canberra""","""ACT"""


In [16]:
election_data = [
    {
        "PollingDistrictIdentifier": electorate["PollingDistrictIdentifier"],
        "PollingPlaces": electorate["PollingPlaces"],
    }
    for electorate in data_dict["MediaFeed"]["Results"]["Election"][0]["House"]["Contests"]["Contest"]
]

In [25]:
(
    pl.DataFrame(election_data[0], strict=False)
    .select(
        pl.col("PollingDistrictIdentifier").struct["Name"].alias("electorate").first(),
        pl.col("PollingDistrictIdentifier").struct["StateIdentifier"].struct["@Id"].alias("state").first(),
        pl.col("PollingPlaces").struct[0].explode().alias("PollingPlace"),
    )
    .select(
        pl.exclude("PollingPlace"),
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Id"].alias("poling_place_id"),
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Name"].alias("poling_place_name"),
        pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Classification"].alias("poling_place_type"),
        pl.col("PollingPlace").struct["FirstPreferences"].struct["Candidate"],
    )
    .explode("Candidate")
    .select(
        pl.exclude("Candidate"),
        pl.col("Candidate").struct["eml:CandidateIdentifier"].struct[0].alias("candidate"),
        pl.col("Candidate").struct["Votes"].struct["@Historic"].alias("votes_past"),
        pl.col("Candidate").struct["Votes"].struct["@Percentage"].alias("vote_percent"),
        pl.col("Candidate").struct["Votes"].struct["@Swing"].alias("vote_swing"),
        pl.col("Candidate").struct["Votes"].struct["#text"].alias("vote_current"),
    )
)

electorate,state,poling_place_id,poling_place_name,poling_place_type,candidate,votes_past,vote_percent,vote_swing,vote_current
str,str,str,str,str,str,str,str,str,str
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28189""","""796""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28336""","""338""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""29373""","""88""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28989""","""742""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""64583""","""Belconnen CANBERRA PPVC""","""PrePollVotingCentre""","""28189""","""395""","""0""","""0""","""0"""
…,…,…,…,…,…,…,…,…,…
"""Canberra""","""ACT""","""32705""","""Woden CANBERRA PPVC""","""PrePollVotingCentre""","""28989""","""5190""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8840""","""Yarralumla""",null,"""28189""","""571""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8840""","""Yarralumla""",null,"""28336""","""216""","""0""","""0""","""0"""


In [26]:
pl.concat(
    [
        pl.DataFrame(election_data[election_index], strict=False)
        .select(
            pl.col("PollingDistrictIdentifier").struct["Name"].alias("electorate").first(),
            pl.col("PollingDistrictIdentifier").struct["StateIdentifier"].struct["@Id"].alias("state").first(),
            pl.col("PollingPlaces").struct[0].explode().alias("PollingPlace"),
        )
        .select(
            pl.exclude("PollingPlace"),
            pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Id"].alias("poling_place_id"),
            pl.col("PollingPlace").struct["PollingPlaceIdentifier"].struct["@Name"].alias("poling_place_name"),
            pl.col("PollingPlace")
            .struct["PollingPlaceIdentifier"]
            .struct["@Classification"]
            .alias("poling_place_type"),
            pl.col("PollingPlace").struct["FirstPreferences"].struct["Candidate"],
        )
        .explode("Candidate")
        .select(
            pl.exclude("Candidate"),
            pl.col("Candidate").struct["eml:CandidateIdentifier"].struct[0].alias("candidate"),
            pl.col("Candidate").struct["Votes"].struct["@Historic"].alias("votes_past"),
            pl.col("Candidate").struct["Votes"].struct["@Percentage"].alias("vote_percent"),
            pl.col("Candidate").struct["Votes"].struct["@Swing"].alias("vote_swing"),
            pl.col("Candidate").struct["Votes"].struct["#text"].alias("vote_current"),
        )
        for election_index, _ in enumerate(election_data)
    ]
)

electorate,state,poling_place_id,poling_place_name,poling_place_type,candidate,votes_past,vote_percent,vote_swing,vote_current
str,str,str,str,str,str,str,str,str,str
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28189""","""796""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28336""","""338""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""29373""","""88""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""8829""","""Barton""",null,"""28989""","""742""","""0""","""0""","""0"""
"""Canberra""","""ACT""","""64583""","""Belconnen CANBERRA PPVC""","""PrePollVotingCentre""","""28189""","""395""","""0""","""0""","""0"""
…,…,…,…,…,…,…,…,…,…
"""Tangney""","""WA""","""11635""","""Winthrop""",null,"""23853""","""0""","""0""","""0""","""0"""
"""Tangney""","""WA""","""11635""","""Winthrop""",null,"""28847""","""105""","""0""","""0""","""0"""
"""Tangney""","""WA""","""11635""","""Winthrop""",null,"""28314""","""282""","""0""","""0""","""0"""
